In [ ]:
%pip install tensorflow
%pip install keras
%pip install Keras-Preprocessing
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/42.6 kB ? eta -:--:--
   -------------------------------------- - 41.0/42.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 41.0/42.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 41.0/42.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 41.0/42.6 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 41.0/42.6 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 42.6/42.6 kB 122.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# import tensorflow as tf
# print("Versão do TensorFlow:", tf.__version__)
# import keras as K
# print("Versão do Keras:", K.__version__)

Versão do TensorFlow: 2.16.1
Versão do Keras: 3.3.3


In [ ]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [ ]:
# Inicializando a Rede Neural Convolucional
classifier = Sequential()

In [ ]:
# Passo 1 - Primeira Camada de Convolução
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

c:\Users\T-GAMER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

In [ ]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))


In [ ]:
# Passo 3 - Flattening
classifier.add(Flatten())

In [ ]:
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
import keras.preprocessing.image as kp

train_datagen = kp.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

validation_datagen = kp.ImageDataGenerator(rescale = 1./255)

AttributeError: module 'keras.preprocessing.image' has no attribute 'ImageDataGenerator'

In [ ]:
# Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('dataset_treino',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_set = validation_datagen.flow_from_directory('dataset_validation',
                                                        target_size = (64, 64),
                                                        batch_size = 32,
                                                        class_mode = 'binary')

In [ ]:
# Executando o treinamento (esse processo pode levar bastante tempo, dependendo do seu computador)
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 5,
                         validation_data = validation_set,
                         validation_steps = 2000)

In [ ]:
# Primeira Imagem
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset_teste/2216.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cachorro'
else:
    prediction = 'Gato'

Image(filename='dataset_teste/2216.jpg')

In [ ]:
# Previsão da primeira imagem
prediction

In [ ]:
# Segunda Imagem
test_image = image.load_img('dataset_teste/2897.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    prediction = 'Cachorro'
else:
    prediction = 'Gato'

Image(filename='dataset_teste/2897.jpg')

In [ ]:
# Previsão da segunda imagem
prediction